In [1]:
import os
import warnings
import sys

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [3]:
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging

In [4]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [5]:
mlflow.set_tracking_uri("http://192.168.7.95:8000/")

In [6]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual,pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [7]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )

    try:
        data = pd.read_csv(csv_url,sep=";")
    except Exception as e:
        logger.exception("Unable to download training and test CSV, check your internet connection. Error: %s", e
                        )

    train, test = train_test_split(data)
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"],axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 0.5
    l1_ratio = 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x,train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y,predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print(" RMSE: %s" % rmse)
        print(" MAE: %s" % mae)
        print(" R2: %s" % r2)

        mlflow.log_param("alpha",alpha)
        mlflow.log_param("l1_ratio",l1_ratio)
        mlflow.log_metric("rmse",rmse)
        mlflow.log_metric("r2",r2)
        mlflow.log_metric("mae",mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store)
        # exit(0)
        input_example = train_x.iloc[[0]]
        if tracking_url_type_store != "file":
            mlflow.sklearn.log_model(lr,"model",
                     registered_model_name="ElasticnetWineModel",input_example=input_example)
        else:
            mlflow.sklearn.log_model(lr,"model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
 RMSE: 0.7931640229276851
 MAE: 0.6271946374319586
 R2: 0.10862644997792614
http


Registered model 'ElasticnetWineModel' already exists. Creating a new version of this model...
2023/09/01 10:59:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 2
Created version '2' of model 'ElasticnetWineModel'.
